# Работа с текстовыми эмбеддингами.

In [1]:
import matplotlib.pyplot as plt
from pathlib import Path
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from sentence_transformers import SentenceTransformer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.neighbors import NearestNeighbors
from catboost import CatBoostClassifier, Pool

# Загрузка данных.

In [2]:
# Путь к папке с датасетом
DATA_DIR = Path("C:/Users/vds/Work/Programming Stuff/ecup/data")

In [3]:
# Загрузка
df_train = pd.read_csv(DATA_DIR / "train.csv", index_col='id')
df_test = pd.read_csv(DATA_DIR / "test_full.csv", index_col='id')

In [4]:
# Обработка данных
import sys
sys.path.append(str(Path.cwd().parent))
from scripts import data_preprocess

In [5]:
import warnings
warnings.filterwarnings('ignore')

df_train_num, df_train_text = data_preprocess.clean_data(df_train)
df_test_num, df_test_text = data_preprocess.clean_data(df_test, type='test')

## Text embeddings

## Анализ и очистка пропусков.

In [6]:
df_train_text.sample(5)

,brand_name,description,name_rus,CommercialTypeName4,ItemID,resolution
id,,,,,,
132470,Xiaomi,ИДЕАЛЬНО СОЧЕТАЮТСЯ С Mesh-СИСТЕМОЙ AX3000. <b...,Wi-Fi роутер Xiaomi Wi-Fi Router AX3200,"Маршрутизатор, точка доступа",64965,0
355045,I-spare,Кабель USB для Applee iPhonee Lightning Type-C...,Кабель USB для Applee iPhonee Lightning Type-C...,Кабель для мобильных устройств,174615,0
443075,HotComputers,<p>Кабель подходит для блока питания Samsung (...,Кабель для блока питания Samsung 5.5*3.0mm,Кабель питания,215556,0
58037,Levsha kaluga,<p>Мы используем только НОВЫЕ комплектующие пр...,Levsha kaluga Системный блок Мощный игровой Пк...,Настольный компьютер,29029,0
414156,Спейс,"Бумага широкоформатная OfficeSpace, 620мм*150м...",Спейс Бумага широкоформатная,Бумага для плоттера,201923,0


In [7]:
print(f"Brand_name unique: {len(df_train['brand_name'].unique())}")
print(f"CommercialTypeName4 unnique: {len(df_train['CommercialTypeName4'].unique())}")

Brand_name unique: 4067
CommercialTypeName4 unnique: 634


In [8]:
for col in df_train_text.columns:
    nans = df_train_text[col].isna().sum()
    percent = nans / len(df_train_text) * 100
    print(f"{col} nans: {nans}, {percent}%")

brand_name nans: 0, 0.0%
description nans: 0, 0.0%
name_rus nans: 0, 0.0%
CommercialTypeName4 nans: 0, 0.0%
ItemID nans: 0, 0.0%
resolution nans: 0, 0.0%


In [9]:
for col in df_test_text.columns:
    nans = df_test_text[col].isna().sum()
    percent = nans / len(df_test_text) * 100
    print(f"{col} nans: {nans}, {percent}%")

brand_name nans: 0, 0.0%
description nans: 0, 0.0%
name_rus nans: 0, 0.0%
CommercialTypeName4 nans: 0, 0.0%
ItemID nans: 0, 0.0%


### 1) Параметры и выбор модели

In [10]:
# Рекомендация для русского/мультиязычного текста: лёгкая и быстрая модель
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
# Альтернатива (чуть тяжелее, обычно лучше качество):
# MODEL_NAME = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"

MAX_LEN = 256     # длина для усечения длинных описаний
BATCH_SIZE = 128  # батчи для кодирования
NORMALIZE = True  # l2-нормализация эмбеддингов (часто улучшает kNN/кластеры)

In [11]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

Device: cuda


### 2) Конструирование текстового представления 

In [12]:
def row_to_text(row):
    # Небольшое «усиление» бренда/названия: ставим их в начало
    return (
        f"[BRAND] {row['brand_name']} || "
        f"[NAME] {row['name_rus']} || "
        f"[TYPE] {row['CommercialTypeName4']} || "
        f"[DESC] {row['description']}"
    ).strip()

texts_train = df_train_text.apply(row_to_text, axis=1).tolist()
texts_test = df_test_text.apply(row_to_text, axis=1).tolist()

### 3) Загрузка и настройка модели

In [13]:
model_transformer = SentenceTransformer(MODEL_NAME, device=DEVICE)

### 4) Получение эмбеддингов. 

In [ ]:
embeddings = model_transformer.encode(
    texts_train,
    batch_size=BATCH_SIZE,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=NORMALIZE,   # L2-нормировка
)

Batches:   0%|          | 0/1541 [00:00<?, ?it/s]

In [15]:
print("Embeddings shape:", embeddings.shape)  # (N, D)

Embeddings shape: (197198, 384)


In [25]:
# Сохраним для последующего переиспользования
os.makedirs("src", exist_ok=True)
np.save("src/text_embeddings.npy", embeddings)

In [15]:
# Загрузим эмбеддинги если запускаем ноутбук заново.
embeddings = np.load("src/text_embeddings.npy")

### 5. Быстрый sanity-check: ближайшие соседи

In [16]:
# Полезно убедиться, что похожие товары действительно рядом в эмбеддинговом пространстве
nn = NearestNeighbors(n_neighbors=5, metric="cosine")
nn.fit(embeddings)
dists, idxs = nn.kneighbors(embeddings[:3])  # смотрим для первых трёх товаров
for i, (di, ii) in enumerate(zip(dists, idxs)):
    print(f"\nТовар {i}:\n", texts_train[i][:200], "...")
    for j, (d, k) in enumerate(zip(di, ii)):
        print(f"  #{j}: id={k}, dist={d:.3f} →", texts_train[k][:120], "...")



Товар 0:
 [BRAND] ACTRUM || [NAME] Мешки для пылесоса PHILIPS TRIATLON, синтетические, многослойные, тип: HR 6947 || [TYPE] Пылесборник || [DESC] Мешки пылесборники для пылесоса PHILIPS, 10 шт., синтетические,  ...
  #0: id=0, dist=0.000 → [BRAND] ACTRUM || [NAME] Мешки для пылесоса PHILIPS TRIATLON, синтетические, многослойные, тип: HR 6947 || [TYPE] Пылесб ...
  #1: id=31198, dist=0.140 → [BRAND] ACTRUM || [NAME] Мешки для пылесосов THOMAS, CAMERON, ZANUSSI, 5 шт + микрофильтр, синтетические, многослойные,  ...
  #2: id=25960, dist=0.141 → [BRAND] ACTRUM || [NAME] Мешки для пылесосов ROWENTA, TEFAL, MOULINEX, синтетические, многослойные || [TYPE] Пылесборник ...
  #3: id=23026, dist=0.166 → [BRAND] ACTRUM || [NAME] Мешки для пылесосов ROWENTA, TEFAL, MOULINEX, 15 шт + 3 микрофильтра, синтетические, многослойн ...
  #4: id=36859, dist=0.166 → [BRAND] ACTRUM || [NAME] Мешки для пылесоса V707, V706, V710, V709, V708, синтетические, многослойные, тип V7D4 || [TYPE ...

Товар 1:
 [BRAND]

## Бейзлайн. Проверка модели с текстами.

In [17]:
# Возьмем числовные данные
num_data = df_train_num.drop(columns=['resolution']).values

# Также возьмем категориальные признаки
cat_cols = ["brand_name", "CommercialTypeName4"]
cat_data = df_train_text[cat_cols].astype(str)

# Теперь объединяем: эмбеддинги + числовые
X_num = np.concatenate([embeddings, num_data], axis=1)  # (N, D + num_features)

# Целевая переменная
y = df_train_num["resolution"].astype(int).values

In [18]:
from sklearn.model_selection import train_test_split

# Трейн/тест сплит
X_train_num, X_val_num, y_train, y_val, train_cat, val_cat = train_test_split(
    X_num, y, cat_data, test_size=0.2, stratify=y, random_state=42
)


In [19]:
X_train = pd.concat(
    [pd.DataFrame(X_train_num), train_cat.reset_index(drop=True)], axis=1
)
X_val = pd.concat(
    [pd.DataFrame(X_val_num), val_cat.reset_index(drop=True)], axis=1
)

In [20]:
# Категориальные признаки теперь — последние len(cat_cols) колонок
cat_features_idx = list(range(X_train_num.shape[1], X_train_num.shape[1] + len(cat_cols)))

train_pool = Pool(X_train, label=y_train, cat_features=cat_features_idx)
val_pool = Pool(X_val, label=y_val, cat_features=cat_features_idx)

In [21]:
# Модель
model = CatBoostClassifier(
    iterations=1000,
    depth=11,
    learning_rate=0.05,
    eval_metric="F1",
    random_seed=42,
    od_type="Iter", 
    od_wait=50,
    task_type="GPU" if torch.cuda.is_available() else "CPU"
)

model.fit(train_pool, eval_set=val_pool,
    verbose=100,
    use_best_model=True,      # Использовать лучшую модель по валидации
    plot=True                 # Построить график обучения
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6767384	test: 0.6713201	best: 0.6713201 (0)	total: 344ms	remaining: 5m 43s
100:	learn: 0.8337413	test: 0.7836257	best: 0.7839419 (97)	total: 29.2s	remaining: 4m 19s
200:	learn: 0.8855510	test: 0.7981504	best: 0.7983109 (199)	total: 56.3s	remaining: 3m 43s
300:	learn: 0.9119462	test: 0.8031369	best: 0.8033810 (297)	total: 1m 22s	remaining: 3m 10s
400:	learn: 0.9312275	test: 0.8051530	best: 0.8053934 (359)	total: 1m 47s	remaining: 2m 40s
500:	learn: 0.9447950	test: 0.8078084	best: 0.8084422 (495)	total: 2m 13s	remaining: 2m 12s
600:	learn: 0.9568566	test: 0.8090927	best: 0.8105390 (597)	total: 2m 38s	remaining: 1m 45s
bestTest = 0.8120481928
bestIteration = 643
Shrink model to first 644 iterations.


In [22]:
# Предсказания
y_pred = model.predict(val_pool)
print("\n=== Classification report ===")
print(classification_report(y_val, y_pred, digits=4))


=== Classification report ===
              precision    recall  f1-score   support

           0     0.9841    0.9906    0.9873     36830
           1     0.8532    0.7747    0.8120      2610

    accuracy                         0.9763     39440
   macro avg     0.9187    0.8826    0.8997     39440
weighted avg     0.9755    0.9763    0.9757     39440



### Подготовка тестовой выборки и сабмит решения.

In [14]:
embeddings_test = model_transformer.encode(
    texts_test,
    batch_size=BATCH_SIZE,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=NORMALIZE,   # L2-нормировка
)
print("Test embeddings shape:", embeddings_test.shape)

Batches:   0%|          | 0/246 [00:00<?, ?it/s]

Test embeddings shape: (31391, 384)


In [16]:
# Сохраним для последующего переиспользования
os.makedirs("src", exist_ok=True)
np.save("src/text_embeddings_test.npy", embeddings_test)

In [23]:
# Скачаем если запускаем заново
embeddings_test = np.load("src/text_embeddings_test.npy")

In [24]:
# Возьмем числовные данные
num_data_test = df_test_num.values

# Также возьмем категориальные признаки
cat_cols = ["brand_name", "CommercialTypeName4"]
cat_data_test = df_test_text[cat_cols].astype(str)

# Теперь объединяем: эмбеддинги + числовые
X_test_num = np.concatenate([embeddings_test, num_data_test], axis=1)  # (N, D + num_features)

In [25]:
# Объединяем в датафрейм для CatBoost
X_test = pd.concat([pd.DataFrame(X_test_num), cat_data_test.reset_index(drop=True)], axis=1)

test_pool = Pool(X_test, cat_features=cat_features_idx)  # cat_features_idx те же, что для валидации

y_pred_test = model.predict(test_pool)

In [26]:
test_predictions = model.predict(test_pool)

submission = pd.DataFrame({
    'id': df_test.index,
    'prediction': test_predictions
})

submission.to_csv('submission.csv', index=False)


print(f"Создан файл submission.csv с {len(submission)} предсказаниями")
print(f"Распределение предсказаний:")
print(submission['prediction'].value_counts())
print()

Создан файл submission.csv с 22760 предсказаниями
Распределение предсказаний:
prediction
0    21541
1     1219
Name: count, dtype: int64



# Image embeddings (+OCR mb)